# GOODRX SCORE

Using this query we are mapping goodrx price with the current mac prices for all the clients on LP/SF. We add claims count and AWP information from the daily totals table. 

Datasets:\
inclusion_table_archive_demo\
goodrx_raw_data\
ytd_combined_claims\
mac_list

In [ ]:
# File Path Definitions
import os

In [ ]:
os.chdir('/home/jupyter/clientpharmacymacoptimization/GER_LP_Code')

In [ ]:
!pip install tqdm

In [ ]:
import pandas as pd
import numpy as np
# import CPMO_parameters as p  
# import CPMO_goodrx_functions as grx
# from CPMO_shared_functions import standardize_df
import BQ
import util_funcs as uf
import re
#Import bigquery
from google.cloud import bigquery
bq = bigquery.Client()
from tqdm import tqdm

In [ ]:
def standardize_df(df):
    '''
    This is a series of common steps to get all dataframes and data inputs into the same
    general format.
    Inputs:
        df - a dataframe that needs to be standardized
    Outputs:
        df - the standardized dataframe
    '''
    del_cols = []
    for col in df.columns:
        if 'Unnamed:' in col:
            del_cols.append(col)
    df = df.drop(columns=del_cols)

    # Add captialization to all names to standardize the input for all clients
    df.columns = map(str.upper, df.columns)
    
    #Captilaize everything in the following columns to make sure that we can use string matching
    # PIP change added CLIENT_NAME
    for column in ['CLIENT', 'BREAKOUT', 'REGION', 'MEASUREMENT', 'CHAIN_GROUP', 'CHAIN_SUBGROUP', 'CLIENT_NAME', 'CUSTOMER_ID', 'MAC_LIST', 'MACLIST', 'VCML_ID']:
        if column in df.columns:
            df[column] = df[column].astype(str)
            df[column] = df[column].apply(lambda x: x.upper())
    
    # Different data sources use different names for Wellcare so this standarized them to "WELLCARE"
    if 'CLIENT' in df.columns:
        df.loc[df.CLIENT == 'WC', 'CLIENT'] = 'WELLCARE'
        # TODO: Is this what we want
        df.CLIENT = df.CLIENT.astype(str)
        
    
    # Ensures that no GPI loses its leading 0
    if 'GPI' in df.columns:
        if df.GPI.dtype == float:
            df.GPI = df.GPI.astype('int64')
            
        df.GPI = df.GPI.astype(str).apply(lambda x: x.split('.')[0])
        df.GPI = ('00' + df.GPI).str[-14:]
    
    # Resolves naming differences of NDC columns between different data sources
    if 'NDC11' in df.columns:
        df.rename(columns={'NDC11': 'NDC'}, inplace=True)
    
    # Ensures that NDCs do not lose leading 0
    if 'NDC' in df.columns:
        if df.NDC.dtype == float:
            df.NDC = df.NDC.astype('int64')
            
        df.NDC = df.NDC.astype(str).apply(lambda x: x.split('.')[0])
        df.NDC = ('0000' + df.NDC).str[-11:]
    
    # Creates a common column of GPI and NDC for string matching
    if ('GPI' in df.columns) & ('NDC' in df.columns):
        df['GPI_NDC'] = df.GPI + '_' + df.NDC
    
    #MARCEL Added. reading client_claim (see Pre_processing.py) from BQ somehow results in object type
    if 'SPEND' in df.columns:
        df.SPEND = df.SPEND.astype('float64')
    if 'AWP' in df.columns:
        df.AWP = df.AWP.astype('float64')
    if 'DRUG_PRICE_AT' in df.columns:
        df.DRUG_PRICE_AT = df.DRUG_PRICE_AT.astype('float64')
    if 'QTY' in df.columns:
        df.QTY = df.QTY.astype('float64')
    # for col in df.columns:
    #     if df[col].dtype == 'object':
    #         assert (df[col].apply(type).value_counts().shape[0] == 1), f'Column "{col}" has mix data types'

    return df

### Identify the clients that are currently on one of the algorithm

In [ ]:
clients = bq.query("""
SELECT
  clients.*
FROM (
  SELECT
      CUSTOMER_ID AS CUSTOMER_ID,
      CLIENT_NAME,
      ALGO
    FROM
      `pbm-mac-lp-prod-ai.pricing_management.inclusion_table`
    WHERE
      REC_CURR_IND = 'Y'
      AND CURRENT_DATE() BETWEEN START_DATE
      AND END_DATE
      AND ALGO IN ('LP', 'SF')
    UNION DISTINCT
    SELECT
      CUSTOMER_ID,
      CLIENT_NAME,
      'LP' AS ALGO
    FROM `anbc-pss-prod.de_gms_enrv_pss_prod.gms_truecost_taxonomy_final`
    WHERE
      CUSTOMER_ID NOT IN (
      SELECT CUSTOMER_ID FROM `pbm-mac-lp-prod-ai.pricing_management.inclusion_table`
      WHERE
        REC_CURR_IND = 'Y'
        AND CURRENT_DATE() BETWEEN START_DATE
        AND END_DATE
        AND ALGO IN ('LP', 'SF'))) clients
INNER JOIN (
  SELECT
    CUSTOMER_ID,
    CLIENT_NAME
  FROM
    `pbm-mac-lp-prod-de.ds_pro_lp.ger_opt_taxonomy_final_TC`) tax
ON
  clients.CUSTOMER_ID = tax.CUSTOMER_ID
""").to_dataframe()

In [ ]:
aetna_clients = bq.query("""
SELECT
  clients.*
FROM (
  SELECT
    CUSTOMER_ID AS CUSTOMER_ID,
    CLIENT_NAME,
    ALGO
  FROM
    `pbm-mac-lp-prod-ai.ds_sandbox.temp_aetna_inclusion_table` 
  WHERE
    REC_CURR_IND = 'Y'
    AND CLIENT_MANAGEMENT_TEAM = 'DS'
    AND CURRENT_DATE() BETWEEN START_DATE
    AND END_DATE) clients
INNER JOIN (
  SELECT
    CUSTOMER_ID,
    CLIENT_NAME
  FROM
    `anbc-pss-prod.de_gms_enrv_pss_prod.gms_aetna_ger_opt_taxonomy_final`) tax
ON
  clients.CUSTOMER_ID = tax.CUSTOMER_ID
  limit 5
""").to_dataframe()

### Prepare the goodrx price data by selecting the goodrx price associated with most commonly dispensed quantity

In [ ]:
raw_goodrx_df = bq.query("SELECT * FROM pbm-mac-lp-prod-de.ds_pro_lp.GOODRX_RAW_DATA").to_dataframe()

raw_goodrx_df = raw_goodrx_df.groupby(['GPI','CHAIN_GROUP','QTY'])\
                             .agg(MIN_GRX_PRC_QTY = ('MIN_GRX_PRC_QTY',min),
                                  MAX_GRX_PRC_QTY = ('MAX_GRX_PRC_QTY',min)).reset_index()

raw_goodrx_df['GOODRX_UNIT_PRICE'] = 0.5*raw_goodrx_df.MAX_GRX_PRC_QTY - \
                                                        (0.5 -1)*raw_goodrx_df.MIN_GRX_PRC_QTY

In [ ]:
ytd_claims_df = standardize_df(bq.query("""
SELECT
MEASUREMENT,
CHAIN_GROUP,
GPI,
SUM(CLAIMS) AS TOTAL_CLAIMS,
SUM(QTY) AS TOTAL_DISP_QTY,
FROM (
  SELECT
    measurement AS MEASUREMENT,
    CASE
      WHEN UPPER(CHAIN_GROUP) LIKE "%CVS%" THEN 'CVS'
      WHEN UPPER(CHAIN_GROUP) LIKE "%KRG%" THEN 'KRG'
      WHEN UPPER(CHAIN_GROUP) LIKE "%WMT%" THEN 'WMT'
      WHEN UPPER(CHAIN_GROUP) LIKE "%WAG%" THEN 'WAG'
      WHEN UPPER(CHAIN_GROUP) LIKE "%RAD%" THEN 'RAD'
      WHEN (UPPER(CHAIN_GROUP) LIKE "%MAIL%")
    OR (UPPER(CHAIN_GROUP) LIKE "%MCHOICE%") THEN "MAIL"
    ELSE
    "OTH"
  END
    AS CHAIN_GROUP,
    CAST(gpi AS STRING) AS GPI,
    CAST(ndc AS STRING) AS NDC,
    CAST(CLAIMS_distinct AS FLOAT64) AS CLAIMS,
    CAST(qty AS FLOAT64) AS QTY,
  FROM
  (SELECT * FROM `anbc-pss-prod.de_gms_enrv_pss_prod.gms_aetna_combined_daily_totals`
  UNION ALL 
  SELECT * FROM `pbm-mac-lp-prod-de.ds_pro_lp.combined_daily_totals`)
  WHERE
      CLAIMS_distinct > 0
    AND MEASUREMENT IN ('R30','R90'))
GROUP BY 1,2,3
""").to_dataframe())

In [ ]:
ytd_claims_df['AVG_QTY_CLAIM'] = ytd_claims_df['TOTAL_DISP_QTY']/ytd_claims_df['TOTAL_CLAIMS'] 

In [ ]:
#Sort the dataframes on QTY for a correct join
raw_goodrx_df = raw_goodrx_df.sort_values('QTY')
ytd_claims_df = ytd_claims_df.sort_values('AVG_QTY_CLAIM')

#Merge on GoodRx QTY equal to less than the avg_qty_claim
ytd_claims_df_grx = pd.merge_asof(ytd_claims_df,
                                  raw_goodrx_df,
                                  left_on = 'AVG_QTY_CLAIM',
                                  right_on= 'QTY',
                                  direction = 'backward',
                                  left_by=['GPI', 'CHAIN_GROUP'],
                                  right_by=['GPI', 'CHAIN_GROUP'],
                                  allow_exact_matches=True)
    
ytd_claims_df_grx_notna = ytd_claims_df_grx[ytd_claims_df_grx.QTY.notna()].reset_index(drop=True)

#For rows that didn't have a backward match, match on the nearest qty
ytd_claims_df_grx_isna = pd.merge_asof(ytd_claims_df.iloc[ytd_claims_df_grx[ytd_claims_df_grx.QTY.isna()].index],
                            raw_goodrx_df,
                            left_on='AVG_QTY_CLAIM',
                            right_on='QTY',
                            direction='nearest',
                            left_by=['GPI',  'CHAIN_GROUP'],
                            right_by=['GPI',  'CHAIN_GROUP'],
                            allow_exact_matches=True).reset_index(drop=True)

ytd_claims_df_grx_raw = ytd_claims_df_grx_notna.append(ytd_claims_df_grx_isna).reset_index(drop=True)

In [ ]:
#This table will store the goodrx prices effective per the latest load. 
#These prices have been selected based on closest merge quantity based on 
#claims data across all the clients and can be used as a benchmark for all clients
#We append the new prices to the existing table such that we have a record of all goodrx prices

grx_raw_df = ytd_claims_df_grx_raw[ytd_claims_df_grx_raw.GOODRX_UNIT_PRICE.notna()]

In [ ]:
grx_raw_df = grx_raw_df[['MEASUREMENT','CHAIN_GROUP','GPI','GOODRX_UNIT_PRICE']]

In [ ]:
grx_raw_df = grx_raw_df.drop_duplicates()

In [ ]:
grx_raw_df['RUN_DATE'] = pd.Timestamp.today().date()

In [ ]:
grx_raw_df.sort_values(['MEASUREMENT','CHAIN_GROUP','GPI','GOODRX_UNIT_PRICE'])

In [ ]:
#Write the resuls to a BQ table

grx_raw_df_schema = [
    bigquery.SchemaField("MEASUREMENT", "STRING"),
    bigquery.SchemaField("CHAIN_GROUP", "STRING"),
    bigquery.SchemaField("GPI", "STRING"),
    bigquery.SchemaField("GOODRX_UNIT_PRICE", "FLOAT"),
    bigquery.SchemaField("RUN_DATE", "DATE") 
]

job_config = bigquery.LoadJobConfig(
                schema = grx_raw_df_schema,
                create_disposition = bigquery.CreateDisposition().CREATE_IF_NEEDED,
                write_disposition = bigquery.WriteDisposition().WRITE_APPEND)
table_id = 'pbm-mac-lp-prod-ai.ds_sandbox.grx_raw_df'
job = bq.load_table_from_dataframe(grx_raw_df, table_id, job_config = job_config)

Map each chain group with the vcml to correctly assign the GoodRx price and MAC price

In [ ]:
vcml_df = standardize_df(bq.query(f"""
SELECT customer_id, 
chnl_ind, 
vcml_id AS CLIENT_VCML_ID 
FROM (SELECT customer_id, 
chnl_ind, 
vcml_id  
FROM `pbm-mac-lp-prod-de.ds_pro_lp.vcml_reference`
WHERE CURRENT_DATE() BETWEEN rec_effective_date AND rec_expiration_date
AND rec_curr_ind = 'Y'
AND CUSTOMER_ID not in (SELECT DISTINCT CUSTOMER_ID FROM `anbc-pss-prod.de_gms_enrv_pss_prod.gms_truecost_taxonomy_final` WHERE current_date() BETWEEN Contract_Eff_Dt and Original_Contract_Exprn_Dt)
UNION DISTINCT 
SELECT customer_id, 
chnl_ind, 
vcml_id  
FROM  `anbc-pss-prod.de_gms_enrv_pss_prod.gms_aetna_vcml_reference`
WHERE CURRENT_DATE() BETWEEN rec_effective_date AND rec_expiration_date
AND rec_curr_ind = 'Y'
UNION DISTINCT 
SELECT customer_id, 
chnl_ind, 
vcml_id  
FROM `pbm-mac-lp-prod-de.ds_pro_lp.vcml_reference_TC`
WHERE CURRENT_DATE() BETWEEN rec_effective_date AND rec_expiration_date
AND rec_curr_ind = 'Y'
AND CUSTOMER_ID in (SELECT DISTINCT CUSTOMER_ID FROM `anbc-pss-prod.de_gms_enrv_pss_prod.gms_truecost_taxonomy_final` WHERE current_date() BETWEEN Contract_Eff_Dt and Original_Contract_Exprn_Dt)
)
-- AND customer_id IN UNNEST(@client_list)
""").to_dataframe())

# remove MAC and the CUSTOMER_ID from the VCML_ID
vcml_df_list = []
# Since the customer IDs may have different lengths, loop through each one and then concat the results
for cid in vcml_df['CUSTOMER_ID'].unique():
    t_vcml_df = vcml_df[vcml_df['CUSTOMER_ID']==cid]
    t_vcml_df['CLIENT_MAC_LIST'] = t_vcml_df['CLIENT_VCML_ID'].str[3+len(cid):]
    vcml_df_list.append(t_vcml_df)
vcml_df = pd.concat(vcml_df_list)

In [ ]:
clients_meas = standardize_df(bq.query(f"""
SELECT CUSTOMER_ID, measurement_rate_category 
    FROM (SELECT DISTINCT CUSTOMER_ID, measurement_rate_category FROM `anbc-pss-prod.fdm_cnfv_pss_prod.gms_ger_opt_customer_info_all_algorithm` 
            WHERE CUSTOMER_ID not in (SELECT DISTINCT CUSTOMER_ID FROM `anbc-pss-prod.de_gms_enrv_pss_prod.gms_truecost_taxonomy_final` WHERE current_date() BETWEEN Contract_Eff_Dt and Original_Contract_Exprn_Dt)
          UNION ALL 
          SELECT DISTINCT CUSTOMER_ID, measurement_rate_category FROM `anbc-pss-prod.de_gms_enrv_pss_prod.gms_aetna_ger_opt_customer_info_all_algorithm`
          UNION ALL 
          SELECT DISTINCT CUSTOMER_ID, measurement_rate_category FROM `anbc-pss-dev.de_gms_enrv_pss_dev.gms_truecost_customer_info_all_algorithm`)
    WHERE measurement_rate_category IN ('R30','M30','R90', 'SRX')
    -- AND CUSTOMER_ID IN UNNEST(@client_list)
    GROUP BY CUSTOMER_ID, measurement_rate_category
""",
job_config = bigquery.QueryJobConfig(
    # query_parameters = [(bigquery.ArrayQueryParameter('client_list', 'STRING', list(clients.CUSTOMER_ID.unique())))]
)).to_dataframe())

In [ ]:
measurement_mapping = standardize_df(bq.query(f"""
SELECT
  CAST(client AS STRING) AS client,
  BREAKOUT,
  REGION,
  -- BG_FLAG,
  MEASUREMENT_CLEAN,
  PREFERRED,
  chain_group AS CHAIN_GROUP,
  chain_subgroup AS CHAIN_SUBGROUP,
  fullawp AS FULLAWP
FROM (
  SELECT client, BREAKOUT, REGION, MEASUREMENT_CLEAN, PREFERRED, chain_group, chain_subgroup, fullawp, 
  --'G' AS BG_FLAG 
  FROM `pbm-mac-lp-prod-de.ds_pro_lp.combined_measurement_mapping`  
  WHERE CLIENT not in (SELECT DISTINCT CUSTOMER_ID FROM `anbc-pss-prod.de_gms_enrv_pss_prod.gms_truecost_taxonomy_final` WHERE current_date() BETWEEN Contract_Eff_Dt and Original_Contract_Exprn_Dt)
  UNION ALL 
  SELECT client, BREAKOUT, REGION, MEASUREMENT_CLEAN, PREFERRED, chain_group, chain_subgroup, fullawp, 
  -- 'G' AS BG_FLAG 
  FROM `anbc-pss-prod.de_gms_enrv_pss_prod.gms_aetna_combined_measurement_mapping` 
  UNION ALL
  SELECT client, BREAKOUT, REGION, MEASUREMENT_CLEAN, PREFERRED, chain_group, chain_subgroup, fullawp, 
  FROM `pbm-mac-lp-prod-de.ds_pro_lp.combined_measurement_mapping_TC`
  WHERE client in (SELECT DISTINCT CUSTOMER_ID FROM `anbc-pss-prod.de_gms_enrv_pss_prod.gms_truecost_taxonomy_final` WHERE current_date() BETWEEN Contract_Eff_Dt and Original_Contract_Exprn_Dt))
-- WHERE CUSTOMER_ID IN UNNEST(@client_list)
""",
job_config = bigquery.QueryJobConfig(
    # query_parameters = [(bigquery.ArrayQueryParameter('client_list', 'STRING', list(clients.CUSTOMER_ID.unique())))]
    )).to_dataframe())

In [ ]:
measurement_mapping[measurement_mapping['CLIENT'] == '586C'].sort_values(['CHAIN_GROUP'])

In [ ]:
query = f"""SELECT 
CUSTOMER_ID,
BIG_CAPPED_PHARMACY_LIST,
SMALL_CAPPED_PHARMACY_LIST,
NON_CAPPED_PHARMACY_LIST,
PSAO_LIST 
    FROM 
    `pbm-mac-lp-prod-ai.pricing_management.clnt_params`
UNION ALL 
SELECT CUSTOMER_ID,
BIG_CAPPED_PHARMACY_LIST,
SMALL_CAPPED_PHARMACY_LIST,
NON_CAPPED_PHARMACY_LIST,
PSAO_LIST
    FROM 
    `anbc-pss-prod.de_gms_enrv_pss_prod.gms_aetna_clnt_params`
UNION ALL
(SELECT
  CONCAT("['", taxonomy.CUSTOMER_ID, "']") AS CUSTOMER_ID,
  BIG_CAPPED_PHARMACY_LIST,
  SMALL_CAPPED_PHARMACY_LIST,
  NON_CAPPED_PHARMACY_LIST,
  PSAO_LIST
FROM (SELECT CUSTOMER_ID, 'a' AS idx FROM `anbc-pss-prod.de_gms_enrv_pss_prod.gms_truecost_taxonomy_final`
  WHERE CUSTOMER_ID NOT IN (SELECT SUBSTRING(CUSTOMER_ID, 3, LENGTH(CUSTOMER_ID)-4) FROM `pbm-mac-lp-prod-ai.pricing_management.clnt_params`)) taxonomy
LEFT JOIN (
  SELECT
    CUSTOMER_ID,
    BIG_CAPPED_PHARMACY_LIST,
    SMALL_CAPPED_PHARMACY_LIST,
    NON_CAPPED_PHARMACY_LIST,
    PSAO_LIST,
    'a' AS idx
  FROM `pbm-mac-lp-prod-ai.pricing_management.clnt_params`
  WHERE SUBSTRING(CUSTOMER_ID, 3, LENGTH(CUSTOMER_ID)-4) IN ('4586')) clnt_param
ON taxonomy.idx = clnt_param.idx)
"""

query_job = bq.query(query)

client_list = [re.sub("[^a-zA-Z0-9]", "", row['CUSTOMER_ID']) for row in query_job]
param = [dict(row) for row in query_job]
records = dict(zip(client_list, param))
print(len(records))
i = 0
clnt_pharm = {}
for client in client_list:
    i+=1
    if i % 100 == 0:
        print(i)
    full_pharm_list = []
    full_pharm_list = list(set(eval(records[client]['BIG_CAPPED_PHARMACY_LIST']) + eval(records[client]['SMALL_CAPPED_PHARMACY_LIST']) +\
                                  eval(records[client]['NON_CAPPED_PHARMACY_LIST']) + eval(records[client]['PSAO_LIST'])))
    full_pharm_list = list(set(np.concatenate([measurement_mapping[(measurement_mapping['CHAIN_GROUP']==cg) & (measurement_mapping.CLIENT == client)]['CHAIN_SUBGROUP'].unique() for cg in full_pharm_list])))
    full_pharm_list = list(set(full_pharm_list) | set(eval(records[client]['BIG_CAPPED_PHARMACY_LIST'])))
    clnt_pharm[client] = full_pharm_list

In [ ]:
query = f"""SELECT CUSTOMER_ID,
CLIENT_TYPE
FROM `pbm-mac-lp-prod-ai.pricing_management.clnt_params`
UNION ALL 
SELECT CUSTOMER_ID,
CLIENT_TYPE
FROM `anbc-pss-prod.de_gms_enrv_pss_prod.gms_aetna_clnt_params`
UNION ALL
(SELECT
  CONCAT("['", taxonomy.CUSTOMER_ID, "']") AS CUSTOMER_ID,
  CLIENT_TYPE,
FROM (SELECT CUSTOMER_ID, 'a' AS idx FROM `anbc-pss-prod.de_gms_enrv_pss_prod.gms_truecost_taxonomy_final`
  WHERE CUSTOMER_ID NOT IN (SELECT SUBSTRING(CUSTOMER_ID, 3, LENGTH(CUSTOMER_ID)-4) FROM `pbm-mac-lp-prod-ai.pricing_management.clnt_params`)) taxonomy
LEFT JOIN (
  SELECT
    CUSTOMER_ID,
    CLIENT_TYPE,
    'a' AS idx
  FROM `pbm-mac-lp-prod-ai.pricing_management.clnt_params`
  WHERE SUBSTRING(CUSTOMER_ID, 3, LENGTH(CUSTOMER_ID)-4) IN ('4586')) clnt_param
ON taxonomy.idx = clnt_param.idx)
"""
query_job = bq.query(query)
client_list = [re.sub("[^a-zA-Z0-9]", "", row['CUSTOMER_ID']) for row in query_job]
client_type = [dict(row) for row in query_job]
client_type_dict = dict(zip(client_list, client_type))    

In [ ]:
pref_pharm_df = pd.DataFrame()

for cid in measurement_mapping.CLIENT.unique():
    if cid in client_list:
        if client_type_dict[cid]['CLIENT_TYPE'] in ['COMMERCIAL','MEDICAID']:
            ##for COMMERCIAL clients, preferred vs. non-preferred does not matter
            pref_pharm_list = measurement_mapping[measurement_mapping.CLIENT == cid][['CLIENT', 'BREAKOUT', 'REGION']].drop_duplicates()
            pref_pharm_list['PREF_PHARM'] = 'None'
        elif client_type_dict[cid]['CLIENT_TYPE'] == 'MEDD':
            if any(measurement_mapping[measurement_mapping.CLIENT == cid].PREFERRED.str.lower().str.startswith('pref')):
                pharm_list = measurement_mapping[measurement_mapping.CLIENT == cid][['CLIENT', 'BREAKOUT', 'REGION']].drop_duplicates()
                pref_pharm_list = measurement_mapping[measurement_mapping.CLIENT == cid].loc[measurement_mapping[measurement_mapping.CLIENT == cid]\
                                                      .PREFERRED.str.lower().str.startswith('pref'), ['CLIENT', 'BREAKOUT', 'REGION', 'CHAIN_SUBGROUP']] \
                                                     .groupby(['CLIENT', 'BREAKOUT', 'REGION'])['CHAIN_SUBGROUP'] \
                                                     .apply(lambda x: ','.join(x.drop_duplicates())) \
                                                     .reset_index() \
                                                     .rename(columns = {'CHAIN_SUBGROUP': 'PREF_PHARM'})
                pref_pharm_list = pharm_list.merge(pref_pharm_list, on = ['CLIENT', 'BREAKOUT', 'REGION'], how = 'left').fillna('None')
            else:
                pref_pharm_list = measurement_mapping[measurement_mapping.CLIENT == cid][['CLIENT', 'BREAKOUT', 'REGION']].drop_duplicates()
                pref_pharm_list['PREF_PHARM'] = 'None'
        else:
            assert False, "CLIENT_TYPE is not of type COMMERCIAL or MEDD."

        pref_pharm_list['PSAO_GUARANTEE'] = 'No'
        pref_pharm_df = pref_pharm_df.append(pref_pharm_list)

In [ ]:
def single_waterfall(wf, vcml_in_df):
        vcml = str(wf[0])
        if len(wf) > 1:
            if any(vcml_in_df.CLIENT_MAC_LIST == vcml):
                return vcml
            else:
                return single_waterfall(wf[1:], vcml_in_df)
        else:
            return vcml

def mac_mapping_waterfall(cmm_df, vcml_in_df, wf, meas, chain_filt, pref_filt):
    waterfall = single_waterfall(wf, vcml_in_df)
    df_filter = (cmm_df['MEASUREMENT'] == meas) & (cmm_df['CHAIN_SUBGROUP'].isin(chain_filt))
    cmm_df.loc[pref_filt & df_filter & (cmm_df.CLIENT_MAC_LIST.isna()), 'CLIENT_MAC_LIST'] = waterfall

In [ ]:
import traceback
errors = []

In [ ]:
all_clients_mac_mapping_columns = ['CUSTOMER_ID','REGION','MEASUREMENT','CHAIN_SUBGROUP','MAC_LIST']
all_clients_mac_mapping = pd.DataFrame(columns = all_clients_mac_mapping_columns)
except_list = []
for cid in tqdm(measurement_mapping.CLIENT.unique()):
# for cid in ['4450', '4412', '5584', '5585', '586C', '4586', '5533', '587C', '4590']:
    # if True:
    try:
        client_measurement_mapping = measurement_mapping[measurement_mapping.CLIENT == cid]    
        if cid in clnt_pharm.keys():
            full_pharm_list = clnt_pharm[cid]
            # print(full_pharm_list)

            MAIL_LIST = []
            if 'MCHOICE' in full_pharm_list:
                full_pharm_list.remove('MCHOICE')  
                MAIL_LIST = ['MCHOICE','MAIL']
            elif 'MCHOICE' not in full_pharm_list:
                MAIL_LIST = ['MAIL']

            # find defaults for mac mapping dictionaries
            hma_mapping = single_waterfall(['11'], vcml_df)
            np_30 = single_waterfall(['199', '1'], vcml_df)
            np_90 = single_waterfall(['399', '3', '199', '1'], vcml_df)
            p_30 = single_waterfall(['177', '1'], vcml_df)
            if (vcml_df.CLIENT_VCML_ID.astype(str).str.contains('MAC'+ cid +'373', regex=False).any()) or (vcml_df.CLIENT_VCML_ID.astype(str).str.contains('TRU'+ cid +'373', regex=False).any()):
                p_90 = single_waterfall(['373', '3', '177', '1'], vcml_df)
            else:
                p_90 = single_waterfall(['377', '3', '177', '1'], vcml_df)
            # the correct is 377 but 373 is a client unique situation
            # small chains that will have EITHER pref OR nonpref VCMLs for Med D -- pick whichever appears
            pbx_30 = single_waterfall(['14', 'P89'], vcml_df)
            pbx_90 = single_waterfall(['42', 'E89'], vcml_df)
            cst_30 = single_waterfall(['17', 'P24'], vcml_df)
            cst_90 = single_waterfall(['47', 'E24'], vcml_df)
            if client_type_dict[cid]['CLIENT_TYPE'] == 'MEDD':
                # Different mappings for AHD, ABS, MJR between Med D & commercial
                abs_30 = single_waterfall(['15', 'P5'], vcml_df)
                abs_90 = single_waterfall(['45', 'E5'], vcml_df)
                ahd_30 = single_waterfall(['16', 'P91'], vcml_df)
                ahd_90 = single_waterfall(['46', 'E91'], vcml_df)
                mjr_90 = '48'
            else:
                abs_30 = single_waterfall(['15', 'P5'], vcml_df)
                abs_90 = single_waterfall(['43', 'E5'], vcml_df)
                ahd_30 = single_waterfall(['16', 'P91'], vcml_df)
                ahd_90 = single_waterfall(['45', 'E91'], vcml_df)
                mjr_90 = '46'





                # mapping dict for big capped and PSAO
            bigcap_psao_map = {'4': ('CVS', 'R30'), '477': ('CVS', 'R30'), '499': ('CVS', 'R30'),
                               '34': ('CVS', 'R90'), '3477': ('CVS', 'R90'), '3499': ('CVS', 'R90'),
                               '5': ('WAG', 'R30'), '577': ('WAG', 'R30'), '599': ('WAG', 'R30'), 'P93': ('WAG', 'R30'),
                               '35': ('WAG', 'R90'), '3577': ('WAG', 'R90'), '3599': ('WAG', 'R90'), 'E93': ('WAG', 'R90'),
                               '6': ('RAD', 'R30'), '677': ('RAD', 'R30'), '699': ('RAD', 'R30'),
                               '36': ('RAD', 'R90'), '3677': ('RAD', 'R90'), '3699': ('RAD', 'R90'),
                               '7': ('WMT', 'R30'), '777': ('WMT', 'R30'), '799': ('WMT', 'R30'), 'P37': ('WMT', 'R30'),
                               '37': ('WMT', 'R90'), '3777': ('WMT', 'R90'), '3799': ('WMT', 'R90'), 'E37': ('WMT', 'R90'),
                               '8': ('KRG', 'R30'), '877': ('KRG', 'R30'), '899': ('KRG', 'R30'), 'P40': ('KRG', 'R30'),
                               'P57': ('KRG', 'R30'),
                               '38': ('KRG', 'R90'), '3877': ('KRG', 'R90'), '3899': ('KRG', 'R90'), 'E40': ('KRG', 'R90'),
                               'E57': ('KRG', 'R90'),
                               '11': ('ACH', 'R30'), '117': ('ACH', 'R30'), '119': ('ACH', 'R30'),
                               '311': ('ACH', 'R90'), '3117': ('ACH', 'R90'), '3119': ('ACH', 'R90'), 
                               '20': ('HMA', 'R90'),
                               '22': ('ART', 'R30'), '227': ('ART', 'R30'), '229': ('ART', 'R30'),
                               '322': ('ART', 'R90'), '3227': ('ART', 'R90'), '3229': ('ART', 'R90'),
                               '33': ('CAR', 'R30'), '337': ('CAR', 'R30'), '339': ('CAR', 'R30'), 
                               '333': ('CAR', 'R90'), '3337': ('CAR', 'R90'), '3339': ('CAR', 'R90'), '30': ('CAR', 'R90'),  
                               '41': ('CVSSP', 'R30'),
                               '91': ('CVSSP', 'R90'),
                               '44': ('ELE', 'R30'), '447': ('ELE', 'R30'), '449': ('ELE', 'R30'),
                               '344': ('ELE', 'R90'), '3447': ('ELE', 'R90'), '3449': ('ELE', 'R90'), '40': ('ELE', 'R90'), 
                               '55': ('EPC', 'R30'), '557': ('EPC', 'R30'), '559': ('EPC', 'R30'),
                               '355': ('EPC', 'R90'), '3557': ('EPC', 'R90'), '3559': ('EPC', 'R90'), '50': ('EPC', 'R90'),
                               '66': ('TPS', 'R30'), '667': ('TPS', 'R30'), '669': ('TPS', 'R30'),
                               '366': ('TPS', 'R90'), '3667': ('TPS', 'R90'), '3669': ('TPS', 'R90'), '60': ('TPS', 'R90'),
                               '14': ('PBX', 'R30'), 'P89': ('PBX', 'R30'),
                               '42': ('PBX', 'R90'), 'E89': ('PBX', 'R90'),
                               '15': ('ABS', 'R30'), 'P5': ('ABS', 'R30'),
                               '43': ('ABS', 'R90'), 'E5': ('ABS', 'R90'),
                               '16': ('AHD', 'R30'), 'P91': ('AHD', 'R30'),
                               '45': ('AHD', 'R90'), 'E91': ('AHD', 'R90'),
                               '18': ('MJR', 'R30'),
                               '46': ('MJR', 'R90'), '48': ('MJR', 'R90'),
                               '54': ('LWD', 'R30'),
                               'E54': ('LWD', 'R90'),
                               '17': ('CST', 'R30'), 'P24': ('CST', 'R30'),
                               '47': ('CST', 'R90'), 'E24': ('CST', 'R90'),
                               '90': ('GIE', 'R30'),
                               'E90': ('GIE', 'R90'),
                               '53': ('HYV', 'R30'),
                               'E53': ('HYV', 'R90'),
                               'P58': ('KIN', 'R30'),
                               'E58': ('KIN', 'R90'),
                               'P11': ('WGS', 'R30'), # Wegmans not Walgreens!
                               'E11': ('WGS', 'R90') 
                               }
            if client_type_dict[cid]['CLIENT_TYPE']=='MEDD':
                bigcap_psao_map.update({
                           '45': ('ABS', 'R90'),
                           '46': ('AHD', 'R90'),
                           '11': ('HMA', 'R30'),
                            '28':('ELE','R30'),
                           'E28':('ELE','R90')})



            #TODO: should not have these hardcoded here -- move to parameters? # move to an input dataset?
            # Both dictionaries (30 and 90) come from Scott Stankey/Dawn Ciullo.  They are current as of Jan 2019 but they can change, both pharmacy names or VCML ID.
            # New chains added Apr 2023 as part of commercial passthrough VCML standaradization process.

            mapping_dictionary_30 = {'ABS': abs_30, 'ACH': '1', 'AHD': ahd_30, 'ART': '22', 'CAR': '33', 'CVS': '4', 'CVSSP': '41', 'ELE': '44',
                                     'EPC': '55', 'GIE': '90', 'HMA': hma_mapping, 'KIN': 'P58', 'KRG': '8', 'NONPREF_OTH': np_30,
                                     'PREF_OTH': p_30, 'RAD': '6', 'SAF': '97', 'TPS': '66', 'WAG': '5', 'WMT': '7', 'LTC': '10', 
                                     'GEN': '1', 'PBX': pbx_30, 'MJR': '18','HYV':'53','CST': cst_30,'AHS':'13', 'WGS': 'P11', 'LWD': '54',
                                     'HVD':'P48', 'HEB' : 'P49', 'PCD': 'P88','MAD':'P94','AMZ':'64','THF':'59','HRT':'61',
                                     'BYD':'81' , 'FCD':'P38' , 'IGD':'P55' , 'KSD':'P57' , 'PCD':'P88' , 'TND' : 'P95' , 'HAD':'P93',
                                     'PUR': '62', 'RUR':'R1','CHD':'C4'}
            mapping_dictionary_90 = {'ABS': abs_90, 'ACH': '3', 'AHD': ahd_90, 'ART': '3', 'CAR': '30', 'CVS': '34', 'CVSSP': '91', 'ELE': '40',
                                     'EPC': '50', 'GIE': 'E90', 'HMA': '20', 'KIN': 'E58', 'KRG': '38', 'NONPREF_OTH': np_90,
                                     'PREF_OTH': p_90, 'RAD': '36', 'SAF': '39', 'TPS': '60', 'WAG': '35', 'WMT': '37', 
                                     'GEN': '3', 'PBX': pbx_90, 'MJR': mjr_90,'HYV':'E53','CST': cst_90, 'WGS': 'E11', 'LWD': 'E54',
                                     'HVD':'E48', 'HEB' : 'E49', 'PCD': 'E88', 'MAD':'E94' , 'AMZ':'E64', 'FCD':'E38' , 'IGD':'E55',
                                     'KSD':'E57' , 'THF':'E59' , 'HRT':'E61' , 'BYD':'E81' , 'PCD':'E88' , 'TND':'E95' , 'HAD':'E93',
                                     'PUR': 'E62', 'RUR':'R3','CHD':'C9'}

            mapping_dictionary_30_xr = dict({key+'_EXTRL': 'XR' for key in mapping_dictionary_30})
            mapping_dictionary_30_xa = dict({key+'_EXTRL': 'XA' for key in mapping_dictionary_30})
            mapping_dictionary_90_xt = dict({key+'_EXTRL': 'XT' for key in mapping_dictionary_90})
            # add R90OK VCMLs
            mapping_dictionary_30.update({key+'_R90OK': 'OK' for key in mapping_dictionary_30})
            mapping_dictionary_90.update({key+'_R90OK': 'OK' for key in mapping_dictionary_90})

            custom_mapping = '88'

            full_pharm_list_x = [p for p in full_pharm_list if p.endswith('_EXTRL')]
            full_pharm_list_nonx = [p for p in full_pharm_list if not p.endswith('_EXTRL')]

            mac_mapping_30 = pd.DataFrame.from_dict({'CHAIN_SUBGROUP': full_pharm_list_nonx, 'MAC_LIST':[mapping_dictionary_30[pharm] for pharm in full_pharm_list_nonx]})
            mac_mapping_30['MEASUREMENT'] = 'R30'
            mac_mapping_90 = pd.DataFrame.from_dict({'CHAIN_SUBGROUP': full_pharm_list_nonx, 'MAC_LIST':[mapping_dictionary_90[pharm] for pharm in full_pharm_list_nonx]})
            mac_mapping_90['MEASUREMENT'] = 'R90'

            mac_mapping_30_xr = pd.DataFrame.from_dict({'CHAIN_SUBGROUP': full_pharm_list_x, 'MAC_LIST':[mapping_dictionary_30_xr[pharm] for pharm in full_pharm_list_x]})
            mac_mapping_30_xr['MEASUREMENT'] = 'R30'
            mac_mapping_30_xa = pd.DataFrame.from_dict({'CHAIN_SUBGROUP': full_pharm_list_x, 'MAC_LIST':[mapping_dictionary_30_xa[pharm] for pharm in full_pharm_list_x]})
            mac_mapping_30_xa['MEASUREMENT'] = 'R30' 
            mac_mapping_90_xt = pd.DataFrame.from_dict({'CHAIN_SUBGROUP': full_pharm_list_x, 'MAC_LIST':[mapping_dictionary_90_xt[pharm] for pharm in full_pharm_list_x]})
            mac_mapping_90_xt['MEASUREMENT'] = 'R90'

            mac_mapping_mail = pd.DataFrame({'CHAIN_SUBGROUP':MAIL_LIST,'MAC_LIST':['2'] * len(MAIL_LIST),'MEASUREMENT':['M30'] * len(MAIL_LIST)})
            mac_mapping_df = pd.concat([mac_mapping_30,mac_mapping_30_xr,mac_mapping_30_xa,mac_mapping_90,mac_mapping_90_xt,mac_mapping_mail]).sort_values('MAC_LIST')
            # del mac_mapping_30,mac_mapping_30_xa,mac_mapping_30_xr,mac_mapping_90_xt,mac_mapping_90,mac_mapping_mail


            client_mac_mapping = mac_mapping_df.copy(deep=True)
            client_mac_mapping['CUSTOMER_ID'] = cid
            client_mac_mapping_temp = client_mac_mapping.copy()
            
            if 'SM30' in measurement_mapping[(measurement_mapping['CLIENT']==cid)]['MEASUREMENT_CLEAN'].unique():
                client_mac_mapping = pd.concat([client_mac_mapping,
                                           pd.DataFrame({'CHAIN_SUBGROUP':['CVS', 'NONPREF_OTH'],
                                                         'MAC_LIST':['SX']*2,
                                                         'MEASUREMENT':['SM30']*2,
                                                        'CUSTOMER_ID': [cid]*2})
                                           ])
#             adding tru to mac mapping
            client_mac_mapping = client_mac_mapping.merge(vcml_df[['CUSTOMER_ID','CLIENT_MAC_LIST', 'CLIENT_VCML_ID']].rename(
                columns = {'CLIENT_VCML_ID': 'VCML_ID', 'CLIENT_MAC_LIST': 'MAC_LIST'}), on = ['CUSTOMER_ID','MAC_LIST'], how ='left')\
            .sort_values('VCML_ID', ascending=False).drop_duplicates(['CHAIN_SUBGROUP', 'MAC_LIST', 'MEASUREMENT', 'CUSTOMER_ID'])
            client_mac_mapping['VCML_ID'] = np.where(client_mac_mapping['VCML_ID'].isna(), 'MAC' + client_mac_mapping['CUSTOMER_ID'].apply(str) + client_mac_mapping['MAC_LIST'].apply(str), client_mac_mapping['VCML_ID'])
            region = cid  # TODO: this should be the region when we loop on region

            pref_pharms = pref_pharm_df['PREF_PHARM'].loc[(pref_pharm_df['REGION']==region) & (pref_pharm_df['BREAKOUT']!=cid+'_MAIL')].unique()
            # if len(pref_pharms) > 0:
            if False:
                pref_pharms = pref_pharms[0].split(',')
            else: 
                pref_pharms = []
            # find vcmls for big capped and psao
            vcml_mac_mapping = vcml_df[vcml_df.CUSTOMER_ID == cid].copy()
            chain_meas_list = vcml_mac_mapping['CLIENT_MAC_LIST'].map(bigcap_psao_map).tolist()

            if pd.isna(chain_meas_list[0]):
                chain_meas_list[0] = (None, None)
            vcml_mac_mapping[['CHAIN_SUBGROUP', 'MEASUREMENT']] = pd.DataFrame(chain_meas_list, index=vcml_mac_mapping.index)
            vcml_mac_mapping = vcml_mac_mapping[pd.notna(vcml_mac_mapping['MEASUREMENT'])]
            vcml_mac_mapping = vcml_mac_mapping[['CUSTOMER_ID', 'MEASUREMENT', 'CHAIN_SUBGROUP', 'CLIENT_MAC_LIST']]  
            vcml_mac_mapping.drop_duplicates(['CUSTOMER_ID', 'MEASUREMENT', 'CHAIN_SUBGROUP'],inplace=True)

            client_vcml_df = vcml_df[vcml_df['CUSTOMER_ID'] == cid]
            client_mac_mapping = client_mac_mapping.merge(client_vcml_df, left_on = ['CUSTOMER_ID','VCML_ID'], right_on=['CUSTOMER_ID','CLIENT_VCML_ID'], how ='left')
            client_mac_mapping.set_index(['CUSTOMER_ID', 'MEASUREMENT', 'CHAIN_SUBGROUP'], inplace=True)
            
            vcml_mac_mapping.set_index(['CUSTOMER_ID', 'MEASUREMENT', 'CHAIN_SUBGROUP'], inplace=True)
            client_mac_mapping['CLIENT_MAC_LIST'].update(vcml_mac_mapping['CLIENT_MAC_LIST'])
            
            client_mac_mapping.reset_index(inplace=True)

            pref_filter = client_mac_mapping['CHAIN_SUBGROUP'].isin(pref_pharms)
            mac_mapping_waterfall(client_mac_mapping, client_vcml_df, ['477', '4', '177', '1'],
                                  'R30', eval(records[cid]['BIG_CAPPED_PHARMACY_LIST']), pref_filter)
            mac_mapping_waterfall(client_mac_mapping, client_vcml_df, ['499', '4', '199', '1'],
                                  'R30', eval(records[cid]['BIG_CAPPED_PHARMACY_LIST']), ~pref_filter)   

            # any missing big chain R90 VCML will: {34,35,36,37,38} -> 9 -> 3 -> {4,5,6,7,8} -> 4 -> 1
            mac_mapping_waterfall(client_mac_mapping, client_vcml_df, ['977', '9', '377', '3', '477', '4', '177', '1'],
                                  'R90', eval(records[cid]['BIG_CAPPED_PHARMACY_LIST']), pref_filter)
            mac_mapping_waterfall(client_mac_mapping, client_vcml_df, ['999', '9', '399', '3', '499', '4', '199', '1'],
                                  'R90', eval(records[cid]['BIG_CAPPED_PHARMACY_LIST']), ~pref_filter)

            ######## waterfall STEP #2 PSAO waterfall
            #R30 PSAOs : {PSAO_R30_vcmls} -> 1
            mac_mapping_waterfall(client_mac_mapping, client_vcml_df, ['177', '1'],
                                  'R30', eval(records[cid]['PSAO_LIST']), pref_filter)

            mac_mapping_waterfall(client_mac_mapping, client_vcml_df, ['199', '1'],
                                  'R30', eval(records[cid]['PSAO_LIST']), ~pref_filter)

            #R90 PSAOs : 3 -> 1
            mac_mapping_waterfall(client_mac_mapping, client_vcml_df, ['377', '3', '177', '1'],
                                  'R90', eval(records[cid]['PSAO_LIST']), pref_filter)
            mac_mapping_waterfall(client_mac_mapping, client_vcml_df, ['399', '3', '199', '1'],
                                  'R90', eval(records[cid]['PSAO_LIST']), ~pref_filter)    

            ALL_OTHER_PHARMACY_LIST = [item for item in eval(records[cid]['SMALL_CAPPED_PHARMACY_LIST']) + eval(records[cid]['NON_CAPPED_PHARMACY_LIST']) if item not in (eval(records[cid]['PSAO_LIST']) + MAIL_LIST)]
            ALL_OTHER_PHARMACY_LIST = list(set(np.concatenate([client_measurement_mapping[client_measurement_mapping['CHAIN_GROUP']==cg]['CHAIN_SUBGROUP'].unique() for cg in ALL_OTHER_PHARMACY_LIST])))

            # print(ALL_OTHER_PHARMACY_LIST)
            #R30: 1
            mac_mapping_waterfall(client_mac_mapping, client_vcml_df, ['177', '1'],
                                  'R30', ALL_OTHER_PHARMACY_LIST, pref_filter)

            mac_mapping_waterfall(client_mac_mapping, client_vcml_df, ['199', '1'],
                                  'R30', ALL_OTHER_PHARMACY_LIST, ~pref_filter)

            #R90 3 -> 1
            mac_mapping_waterfall(client_mac_mapping, client_vcml_df, ['377', '3', '177', '1'],
                                  'R90', ALL_OTHER_PHARMACY_LIST, pref_filter)
            mac_mapping_waterfall(client_mac_mapping, client_vcml_df, ['399', '3', '199', '1'],
                                  'R90', ALL_OTHER_PHARMACY_LIST, ~pref_filter)

            if not any(measurement_mapping[measurement_mapping.CLIENT == cid]['MEASUREMENT_CLEAN'].isin(['R90','R90P','R90p','R90N','R90n'])):
                client_mac_mapping = client_mac_mapping[client_mac_mapping['MEASUREMENT'].apply(lambda x:x[:3]) != 'R90']
                
            if 'SR30' in measurement_mapping[(measurement_mapping['CLIENT']==cid)]['MEASUREMENT_CLEAN'].unique():
                client_mac_mapping_temp = client_mac_mapping[client_mac_mapping['MEASUREMENT'].str.contains('R')].copy()
                client_mac_mapping_temp['MEASUREMENT'] = 'S' + client_mac_mapping_temp['MEASUREMENT']
                client_mac_mapping = pd.concat([client_mac_mapping, client_mac_mapping_temp])

            if pref_pharms == []:
                client_mac_mapping = client_mac_mapping[client_mac_mapping['CHAIN_SUBGROUP'] != 'PREF_OTH'] 
            if client_mac_mapping[client_mac_mapping['CHAIN_SUBGROUP']=='MAIL']['CLIENT_VCML_ID'].isna().all():
                client_mac_mapping = client_mac_mapping[client_mac_mapping['CHAIN_SUBGROUP'] != 'MAIL']    
            client_mac_mapping['CLIENT_VCML_ID'] = client_mac_mapping['CUSTOMER_ID'] + client_mac_mapping['CLIENT_MAC_LIST']
            #REGION MAPPING
            client_mac_mapping['REGION'] = client_mac_mapping['CUSTOMER_ID']

            # drop, reorder, and rename columns
            client_mac_mapping = client_mac_mapping[['CUSTOMER_ID','REGION','MEASUREMENT','CHAIN_SUBGROUP','CLIENT_VCML_ID']]\
                        .rename(columns={'CLIENT_VCML_ID':'MAC_LIST'})
            client_mac_mapping_temp = client_mac_mapping.copy()
            client_vcml_df_temp = client_vcml_df.copy()
            
            client_vcml_df['MAC_LIST'] = client_vcml_df.CUSTOMER_ID + client_vcml_df.CLIENT_MAC_LIST
            client_mac_mapping = client_mac_mapping.merge(client_vcml_df[['CUSTOMER_ID','MAC_LIST', 'CLIENT_VCML_ID']], on = ['CUSTOMER_ID','MAC_LIST'], how ='left').sort_values('CLIENT_VCML_ID', ascending=False).drop_duplicates(['CHAIN_SUBGROUP', 'MAC_LIST', 'MEASUREMENT', 'CUSTOMER_ID'])
            
            client_vcml_df.drop(columns = ['MAC_LIST'], inplace=True)
            client_mac_mapping['CLIENT_VCML_ID'] = np.where(client_mac_mapping['CLIENT_VCML_ID'].isna(), 'MAC' + client_mac_mapping.MAC_LIST, client_mac_mapping['CLIENT_VCML_ID'])
            
            # client_mac_mapping['CLIENT_VCML_ID'] = 'MAC' + client_mac_mapping.MAC_LIST
            mac_mapping = client_mac_mapping.merge(client_vcml_df, how = 'inner', on = ['CUSTOMER_ID','CLIENT_VCML_ID'])
            mac_mapping['RUN_DATE'] = pd.Timestamp.today().date()
            mac_mapping = mac_mapping[['CUSTOMER_ID', 'REGION', 'MEASUREMENT', 'CHAIN_SUBGROUP', 'CLIENT_VCML_ID', 'MAC_LIST','RUN_DATE']]
            
            all_clients_mac_mapping = pd.concat([all_clients_mac_mapping,mac_mapping])

    except:
        except_list.append(cid)

In [ ]:
all_clients_mac_mapping[all_clients_mac_mapping['CUSTOMER_ID']=='4412']

In [ ]:
for i in list(set(errors)):
    print(i)
    # print()

In [ ]:
all_clients_mac_mapping.loc[:,'CHAIN_GROUP'] = all_clients_mac_mapping.loc[:,'CHAIN_SUBGROUP']
all_clients_mac_mapping.loc[all_clients_mac_mapping.CHAIN_SUBGROUP == 'CVSSP','CHAIN_GROUP'] = 'CVS'
all_clients_mac_mapping = all_clients_mac_mapping[['CUSTOMER_ID', 'REGION', 'MEASUREMENT', 'CHAIN_SUBGROUP', 'CHAIN_GROUP', 'MAC_LIST','CLIENT_VCML_ID', 'RUN_DATE']]

In [ ]:
all_clients_mac_mapping.CUSTOMER_ID.nunique()

In [ ]:
len(except_list)

In [ ]:
# #Write the resuls to a BQ table

grx_mac_mapping_schema = [
    bigquery.SchemaField("CUSTOMER_ID", "STRING"),
    bigquery.SchemaField("REGION", "STRING"),
    bigquery.SchemaField("MEASUREMENT", "STRING"),
    bigquery.SchemaField("CHAIN_SUBGROUP", "STRING"),
    bigquery.SchemaField("CHAIN_GROUP", "STRING"),
    bigquery.SchemaField("MAC_LIST", "STRING"),  
    bigquery.SchemaField("CLIENT_VCML_ID", "STRING"),
    bigquery.SchemaField("RUN_DATE", "DATE")   
]

job_config = bigquery.LoadJobConfig(
                schema = grx_mac_mapping_schema,
                create_disposition = bigquery.CreateDisposition().CREATE_IF_NEEDED,
                write_disposition = bigquery.WriteDisposition().WRITE_APPEND)
table_id = 'pbm-mac-lp-prod-ai.ds_sandbox.grx_mac_mapping'
job = bq.load_table_from_dataframe(all_clients_mac_mapping, table_id, job_config = job_config)

### Run this query in BQ (preferred) and append the results to vcml_grx_mac

In [ ]:
q = """
CREATE OR REPLACE TABLE `pbm-mac-lp-prod-ai.ds_sandbox.nadac_test_eva` AS (
WITH
  MAC_MAPPING AS (
  SELECT
    *
  FROM
    `pbm-mac-lp-prod-ai.ds_sandbox.grx_mac_mapping`
  WHERE RUN_DATE = (SELECT MAX(RUN_DATE) FROM `pbm-mac-lp-prod-ai.ds_sandbox.grx_mac_mapping`)
),

NONSPCLT_CLAIMS AS (
  SELECT
    *
  FROM (
    SELECT
      CUSTOMER_ID,
      GPI,
      NDC
    FROM
      (SELECT * FROM `pbm-mac-lp-prod-de.ds_pro_lp.combined_daily_totals`
       UNION DISTINCT 
       SELECT * FROM `anbc-pss-prod.de_gms_enrv_pss_prod.gms_aetna_standard_combined_daily_totals`)
    WHERE
      CUSTOMER_ID IN (
      SELECT
        DISTINCT CUSTOMER_ID
      FROM
        MAC_MAPPING)
        AND GNRCIND = 'TRUE'
        AND customer_id not in (SELECT DISTINCT CUSTOMER_ID FROM `anbc-pss-prod.de_gms_enrv_pss_prod.gms_truecost_taxonomy_final` WHERE current_date() BETWEEN Contract_Eff_Dt and Original_Contract_Exprn_Dt)
    GROUP BY
      CUSTOMER_ID,
      GPI,
      NDC)),

TRUECOST_CLAIMS AS (
  SELECT
    *
  FROM (
    SELECT
      CUSTOMER_ID,
      GPI,
      NDC
    FROM
      `pbm-mac-lp-prod-de.ds_pro_lp.combined_daily_totals_TC`
    WHERE
      CUSTOMER_ID IN (
      SELECT
        DISTINCT CUSTOMER_ID
      FROM
        MAC_MAPPING)
    AND BG_FLAG = 'G'
    AND customer_id in (SELECT DISTINCT CUSTOMER_ID FROM `anbc-pss-prod.de_gms_enrv_pss_prod.gms_truecost_taxonomy_final` WHERE current_date() BETWEEN Contract_Eff_Dt and Original_Contract_Exprn_Dt)
    GROUP BY
      CUSTOMER_ID,
      GPI,
      NDC)),

SPCLT_CLAIMS AS (SELECT 
  CLCODE AS CUSTOMER_ID, 
  clms.GPI, 
  clms.NDC, 
FROM `anbc-pss-prod.de_gms_enrv_pss_prod.gms_spec_client_pharma_clms` clms
WHERE
      CLCODE IN (
      SELECT
        DISTINCT CUSTOMER_ID
      FROM
        MAC_MAPPING)
AND CLCODE not in (SELECT DISTINCT CUSTOMER_ID FROM `anbc-pss-prod.de_gms_enrv_pss_prod.gms_truecost_taxonomy_final` WHERE current_date() BETWEEN Contract_Eff_Dt and Original_Contract_Exprn_Dt)

GROUP BY 1,2,3
),

CLAIMS AS (SELECT * FROM NONSPCLT_CLAIMS UNION ALL SELECT * FROM SPCLT_CLAIMS UNION ALL SELECT * FROM TRUECOST_CLAIMS),

claims_mapped AS (
         SELECT * FROM
            CLAIMS),


NADAC_WAC AS (
  SELECT CUSTOMER_ID, GPI, NDC, COALESCE(NADAC, WAC) AS NADAC_UNIT_PRICE FROM
(SELECT * FROM `pbm-mac-lp-prod-de.ds_pro_lp.nadac_wac_price`
where CUSTOMER_ID not in (SELECT DISTINCT CUSTOMER_ID FROM `anbc-pss-prod.de_gms_enrv_pss_prod.gms_truecost_taxonomy_final` WHERE current_date() BETWEEN Contract_Eff_Dt and Original_Contract_Exprn_Dt) 
UNION ALL
SELECT * FROM `pbm-mac-lp-prod-de.ds_pro_lp.nadac_wac_price_TC`
where CUSTOMER_ID in (SELECT DISTINCT CUSTOMER_ID FROM `anbc-pss-prod.de_gms_enrv_pss_prod.gms_truecost_taxonomy_final` WHERE current_date() BETWEEN Contract_Eff_Dt and Original_Contract_Exprn_Dt) )
WHERE BRND_GNRC_CD = 'GNRC'
),

NADAC_ALL_CLIENTS AS (
SELECT distinct GPI, NDC,
    PERCENTILE_DISC(NADAC_UNIT_PRICE, 0.50) OVER(PARTITION BY GPI, NDC) AS NADAC_UNIT_PRICE
FROM NADAC_WAC
)

-- NADAC_TABLE AS (
  
SELECT claims_filtered.*, 
  COALESCE(NADAC_NDC.NADAC_UNIT_PRICE, NADAC_GPI.NADAC_UNIT_PRICE, NADAC_NDC_ALL.NADAC_UNIT_PRICE, NADAC_GPI_ALL.NADAC_UNIT_PRICE) AS NADAC_UNIT_PRICE 
  -- NADAC_NDC.NADAC_UNIT_PRICE,
  -- NADAC_GPI.NADAC_UNIT_PRICE,
  FROM (SELECT DISTINCT CUSTOMER_ID, GPI, NDC FROM claims_mapped) claims_filtered
LEFT JOIN (SELECT CUSTOMER_ID, GPI, NDC, NADAC_UNIT_PRICE AS NADAC_UNIT_PRICE
              FROM NADAC_WAC
              WHERE NDC not like '%*%') NADAC_NDC
    ON claims_filtered.GPI = NADAC_NDC.GPI
    AND claims_filtered.NDC = NADAC_NDC.NDC
    AND claims_filtered.CUSTOMER_ID = NADAC_NDC.CUSTOMER_ID
  LEFT JOIN (SELECT CUSTOMER_ID, GPI, NADAC_UNIT_PRICE AS NADAC_UNIT_PRICE
              FROM NADAC_WAC
              WHERE NDC like '%*%') NADAC_GPI
    ON claims_filtered.GPI = NADAC_GPI.GPI
    AND claims_filtered.CUSTOMER_ID = NADAC_GPI.CUSTOMER_ID
  LEFT JOIN (SELECT GPI, NDC, NADAC_UNIT_PRICE
              FROM NADAC_ALL_CLIENTS
              WHERE NDC not like '%*%') NADAC_NDC_ALL
    ON claims_filtered.GPI = NADAC_NDC_ALL.GPI
    AND claims_filtered.NDC = NADAC_NDC_ALL.NDC
  LEFT JOIN (SELECT GPI, NADAC_UNIT_PRICE
              FROM NADAC_ALL_CLIENTS
              WHERE NDC like '%*%') NADAC_GPI_ALL
    ON claims_filtered.GPI = NADAC_GPI_ALL.GPI
    );
-- INSERT INTO `pbm-mac-lp-prod-ai.ds_sandbox.vcml_grx_mac_archive` 
-- (
WITH
  MAC_MAPPING AS (
  SELECT
    *
  FROM
    `pbm-mac-lp-prod-ai.ds_sandbox.grx_mac_mapping`
  WHERE RUN_DATE = (SELECT MAX(RUN_DATE) FROM `pbm-mac-lp-prod-ai.ds_sandbox.grx_mac_mapping`)
),

NONSPCLT_CLAIMS AS (
  SELECT
    *,
    CASE
      WHEN TOT_QTY > 0 THEN TOT_AWP/TOT_QTY
    ELSE
    0
  END
    AS AWP_UNIT,
  FROM (
    SELECT
      CUSTOMER_ID,
      REGION,
      MEASUREMENT,
      CHAIN_SUBGROUP,
      CHAIN_GROUP,
      GPI,
      NDC,
      -- CASE WHEN GNRCIND = 'TRUE' THEN 'G' ELSE 'B' END AS BG_FLAG,
      SUM(CLAIMS_distinct) AS CLAIMS,
      SUM(AWP) AS TOT_AWP,
      SUM(QTY) AS TOT_QTY,
      SUM(SPEND) AS TOT_SPEND,
      SUM(DISP_FEE) AS TOT_DISP_FEE,
      SUM(MEMBER_COST) AS TOT_MBR_COST
    FROM
      (SELECT * FROM `pbm-mac-lp-prod-de.ds_pro_lp.combined_daily_totals`
       UNION DISTINCT 
       SELECT * FROM `anbc-pss-prod.de_gms_enrv_pss_prod.gms_aetna_standard_combined_daily_totals`)
    WHERE
      CUSTOMER_ID IN (
      SELECT
        DISTINCT CUSTOMER_ID
      FROM
        MAC_MAPPING)
        AND GNRCIND = 'TRUE'
        AND customer_id not in (SELECT DISTINCT CUSTOMER_ID FROM `anbc-pss-prod.de_gms_enrv_pss_prod.gms_truecost_taxonomy_final` WHERE current_date() BETWEEN Contract_Eff_Dt and Original_Contract_Exprn_Dt)
    GROUP BY
      CUSTOMER_ID,
      REGION, 
      MEASUREMENT,
      CHAIN_SUBGROUP,
      CHAIN_GROUP,
      GPI,
      NDC)
      
  WHERE
    TOT_AWP > 0
    ),

TRUECOST_CLAIMS AS (
  SELECT
    *,
    CASE
      WHEN TOT_QTY > 0 THEN TOT_AWP/TOT_QTY
    ELSE
    0
  END
    AS AWP_UNIT,
  FROM (
    SELECT
      CUSTOMER_ID,
      REGION,
      MEASUREMENT,
      CHAIN_SUBGROUP,
      CHAIN_GROUP,
      GPI,
      NDC,
      -- BG_FLAG,
      SUM(CLAIMS_distinct) AS CLAIMS,
      SUM(AWP) AS TOT_AWP,
      SUM(QTY) AS TOT_QTY,
      SUM(SPEND) AS TOT_SPEND,
      SUM(DISP_FEE) AS TOT_DISP_FEE,
      SUM(MEMBER_COST) AS TOT_MBR_COST
    FROM
      `pbm-mac-lp-prod-de.ds_pro_lp.combined_daily_totals_TC`
    WHERE
      CUSTOMER_ID IN (
      SELECT
        DISTINCT CUSTOMER_ID
      FROM
        MAC_MAPPING)
    AND BG_FLAG = 'G'
    AND customer_id in (SELECT DISTINCT CUSTOMER_ID FROM `anbc-pss-prod.de_gms_enrv_pss_prod.gms_truecost_taxonomy_final` WHERE current_date() BETWEEN Contract_Eff_Dt and Original_Contract_Exprn_Dt)
    GROUP BY
      CUSTOMER_ID,
      REGION, 
      MEASUREMENT,
      CHAIN_SUBGROUP,
      CHAIN_GROUP,
      GPI,
      NDC)
  WHERE
    TOT_AWP > 0),

awp_hist AS (SELECT GPI, NDC, MIN(DRUG_PRICE_AT) AS AWP_UNIT
FROM `pbm-mac-lp-prod-de.ds_pro_lp.awp_history_table`
GROUP BY 1,2),


SPCLT_CLAIMS AS (SELECT 
  CLCODE AS CUSTOMER_ID, 
  CLCODE AS REGION,
  'SPCLT' AS MEASUREMENT,
  'CMK_SPECIALTY' AS CHAIN_SUBGROUP,
  'CMK_SPECIALTY' AS CHAIN_GROUP,
  clms.GPI, 
  clms.NDC, 
  -- 'G' AS BG_FLAG,
  SUM(ROUND(CAST(claim_count AS FLOAT64),0)) AS CLAIMS,
  SUM(ROUND(CAST(AWP AS FLOAT64),4)) AS TOT_AWP,
  SUM(ROUND(CAST(QUANTITY AS FLOAT64),4)) AS TOT_QTY,
  SUM(ROUND(CAST(INGRED_COST AS FLOAT64),4)) AS TOT_SPEND,
  0 AS TOT_DISP_FEE,
  SUM(ROUND(CAST(member_cost AS FLOAT64),4)) AS TOT_MBR_COST,
  MIN(AWP_UNIT) AS AWP_UNIT
FROM `anbc-pss-prod.de_gms_enrv_pss_prod.gms_spec_client_pharma_clms` clms
LEFT JOIN awp_hist 
 ON awp_hist.GPI = clms.GPI
 AND awp_hist.NDC = clms.NDC
WHERE
      CLCODE IN (
      SELECT
        DISTINCT CUSTOMER_ID
      FROM
        MAC_MAPPING)
AND CLCODE not in (SELECT DISTINCT CUSTOMER_ID FROM `anbc-pss-prod.de_gms_enrv_pss_prod.gms_truecost_taxonomy_final` WHERE current_date() BETWEEN Contract_Eff_Dt and Original_Contract_Exprn_Dt)

GROUP BY 1,2,3,4,5,6,7
),

CLAIMS AS (SELECT * FROM NONSPCLT_CLAIMS UNION ALL SELECT * FROM SPCLT_CLAIMS UNION ALL SELECT * FROM TRUECOST_CLAIMS),

client_info AS (
  SELECT
      CUSTOMER_ID,
      CLIENT_NAME,
      CLIENT_TYPE,
      guarantee_category,
      CASE
        WHEN guarantee_category LIKE "MedD%" THEN "MEDD"
      ELSE
      'COMMERCIAL'
    END
      AS MEDD_COMM
    FROM
      (SELECT       CUSTOMER_ID,
      CLIENT_NAME,
      CLIENT_TYPE,
      guarantee_category FROM (SELECT CUSTOMER_ID,
            CLIENT_NAME,
            CLIENT_TYPE,
            'TrueCost' AS guarantee_category
            FROM  `anbc-pss-prod.de_gms_enrv_pss_prod.gms_truecost_taxonomy_final`
            WHERE current_date() BETWEEN Contract_Eff_Dt and Original_Contract_Exprn_Dt
            UNION DISTINCT 
            SELECT
            CUSTOMER_ID,
                  CLIENT_NAME,
                  CLIENT_TYPE,
                  guarantee_category FROM `pbm-mac-lp-prod-de.landing.GER_OPT_TAXONOMY_FINAL`
                  WHERE CUSTOMER_ID not in (SELECT DISTINCT CUSTOMER_ID FROM `anbc-pss-prod.de_gms_enrv_pss_prod.gms_truecost_taxonomy_final` WHERE current_date() BETWEEN Contract_Eff_Dt and Original_Contract_Exprn_Dt))
      UNION DISTINCT 
      SELECT       CUSTOMER_ID,
      CLIENT_NAME,
      CLIENT_TYPE,
      guarantee_category FROM `anbc-pss-prod.de_gms_enrv_pss_prod.gms_aetna_ger_opt_taxonomy_final`)
    WHERE
      CUSTOMER_ID IN (
      SELECT
        DISTINCT CUSTOMER_ID
      FROM
        MAC_MAPPING)
),

full_mac AS (
  SELECT GPI,
      NDC,
      MAC_LIST,
      MAX(MAC) AS MAC,
    AVG(PRICE) AS PRICE 
    FROM (SELECT
    DISTINCT
      GPI,
      NDC,
      MAC_LIST,
      MAC,
      -- 'G' AS BG_FLAG,
      PRICE
      FROM
      (SELECT * FROM `pbm-mac-lp-prod-de.ds_pro_lp.mac_list_TC` 
        UNION DISTINCT 
        SELECT * FROM `anbc-pss-prod.de_gms_enrv_pss_prod.gms_aetna_mac_list`
        )
    UNION DISTINCT
    SELECT
      GPI,
      NDC,
      MAC_LIST,
      MAC,
      -- BG_FLAG,
      PRICE
    FROM
      (SELECT GPI,
      NDC,
      MAC_LIST,
      MAC,
      -- 'G' AS BG_FLAG,
      GENERIC_PRICE AS PRICE FROM `pbm-mac-lp-prod-de.ds_pro_lp.tru_price_list_TC` 
      where GENERIC_PRICE > 0
        -- UNION DISTINCT 
--         SELECT GPI,
--       NDC,
--       MAC_LIST,
--       MAC,
--       'B' AS BG_FLAG,
--       BRAND_PRICE AS PRICE FROM `pbm-mac-lp-prod-de.ds_pro_lp.tru_price_list_TC` 
--       where BRAND_PRICE >0
        )
    )
    GROUP BY GPI, NDC, MAC_LIST
),

mac_ndc AS (SELECT
            GPI,
            NDC,
            MAC_LIST,
            PRICE
          FROM
            full_mac),

mac_gpi AS (SELECT
          GPI,
          NDC,
          MAC_LIST,
          PRICE
        FROM
          full_mac
        WHERE
          NDC LIKE "%*%"),

claims_mapped AS (
         SELECT * EXCEPT(RUN_DATE), MAX(RUN_DATE) OVER() AS RUN_DATE,
         FROM( 
          SELECT
            CLAIMS.*,
            MAC_LIST,
            RUN_DATE
          FROM
            CLAIMS
          LEFT JOIN
            MAC_MAPPING
          ON
            CLAIMS.CUSTOMER_ID = MAC_MAPPING.CUSTOMER_ID
            AND CLAIMS.MEASUREMENT = MAC_MAPPING.MEASUREMENT
            AND CLAIMS.CHAIN_SUBGROUP = MAC_MAPPING.CHAIN_SUBGROUP
          WHERE
            (MAC_MAPPING.CLIENT_VCML_ID IS NOT NULL OR CLAIMS.CHAIN_GROUP = 'CMK_SPECIALTY' or claims.MEASUREMENT like 'S%'))),

drug_data_ndc AS (
  SELECT
      gpi_nm,
      GPI,
      NDC,
      drug_multi_src_cd,
      IFNULL(spclt_cd, 'N') AS spclt_drug_ind,
      drg.eff_dt,
      drg.exprn_dt
  FROM (
      SELECT
        gpi_nm,
        gpi_cd AS GPI,
        drug_id AS NDC,
        drug_multi_src_cd,
        eff_dt,
        exprn_dt
      FROM `anbc-pss-prod.fdm_cnfv_pss_prod.gms_v_drug_denorm_gcp`
      WHERE gpi_cd  NOT LIKE "% %"
        AND gpi_nm != " "
        ) drg
  LEFT JOIN (
      SELECT
        gpi_cd, 
        drug_id,
        eff_dt,
        exprn_dt,
        'Y' AS spclt_cd
      FROM `anbc-pss-prod.fdm_cnfv_pss_prod.gms_v_drug_denorm_gcp`
      WHERE spclt_ctgry_cd IN ('B','L','H','T','V','R','P')
        AND CONCAT(spclt_offr_cd, spclt_chrct_cd) IN ('101','001','002','003','004','007','008','009','011')
        AND UPPER(spclt_drug_ind) = 'Y'
        AND gpi_cd NOT IN ('',
                      '6240502500E520',
                      '21101040102110',
                      '86101070002005',
                      '12353077100120',
                      '3950004010E520')) spl
  ON drg.gpi = spl.gpi_cd
      AND drg.ndc = spl.drug_id
      AND drg.eff_dt = spl.eff_dt
      AND drg.exprn_dt = spl.exprn_dt
),

spclt AS (
  SELECT
      GPI,
      MIN(gpi_nm) AS gpi_nm,
      MAX(spclt_drug_ind) AS SPCLT_DRUG_IND,
    FROM drug_data_ndc
    WHERE CURRENT_DATE('America/New_York') BETWEEN eff_dt AND exprn_dt
    GROUP BY 1
),

vcml_ref_spec AS (
SELECT CUSTOMER_ID, vcml_id AS MAC_LIST ,SUBSTRING(VCML_ID, 4+LENGTH(CUSTOMER_ID)) AS VCML_IND
FROM (SELECT customer_id, vcml_id  
FROM `anbc-pss-prod.fdm_cnfv_pss_prod.gms_v_ger_opt_vcml_reference_lp`
WHERE CURRENT_DATE() BETWEEN rec_effective_date AND rec_expiration_date
AND rec_curr_ind = 'Y'
UNION DISTINCT 
SELECT customer_id, vcml_id  
FROM  `anbc-pss-prod.de_gms_enrv_pss_prod.gms_aetna_vcml_reference`
WHERE CURRENT_DATE() BETWEEN rec_effective_date AND rec_expiration_date
AND rec_curr_ind = 'Y')
WHERE SUBSTRING(VCML_ID, 4+LENGTH(CUSTOMER_ID)) IN ('41','4','1','SX','S3','S9')),

allmacs_spec AS (
SELECT * FROM (
SELECT CUSTOMER_ID, GPI, NDC, vcml_ref_spec.VCML_IND, PRICE AS CURRENT_MAC_PRICE 
FROM full_mac
INNER JOIN vcml_ref_spec
ON full_mac.MAC = vcml_ref_spec.MAC_LIST)
PIVOT(MIN(CURRENT_MAC_PRICE) FOR VCML_IND IN ('SX','S3','S9','41','4','1'))),

curr_mac_spec AS (
SELECT Spec_CLAIMS.*, 
COALESCE(allmacs_spec.SX,allmacs_spec.S3,allmacs_spec.S9,LEAST(allmacs_spec.41,allmacs_spec.4,allmacs_spec.1),allmacs_gpi.SX,allmacs_gpi.S3,allmacs_gpi.S9,LEAST(allmacs_gpi.41,allmacs_gpi.4,allmacs_gpi.1)) AS CURR_MAC_PRC,
FROM (SELECT * FROM SPCLT_CLAIMS
    ) Spec_CLAIMS
LEFT JOIN allmacs_spec
ON Spec_CLAIMS.CUSTOMER_ID = allmacs_spec.CUSTOMER_ID
AND Spec_CLAIMS.GPI = allmacs_spec.GPI
AND Spec_CLAIMS.NDC = allmacs_spec.NDC
LEFT JOIN (SELECT * FROM allmacs_spec WHERE NDC LIKE "%*%") allmacs_gpi
ON Spec_CLAIMS.CUSTOMER_ID = allmacs_gpi.CUSTOMER_ID
AND Spec_CLAIMS.GPI = allmacs_gpi.GPI),

COST_PLUS_TABLE AS (
SELECT
  GPI_CD AS GPI,
  ROUND(MAX(CAST(REPLACE(REPLACE(TRIM(_unit_price_with_shipping_), '$', ''), ',', '') AS FLOAT64)),4) AS MCCP_UNIT_PRICE,
  -- CASE WHEN BRND_GNRC_CD = 'GNRC' THEN 'G'
  -- ELSE 'B' END AS BG_FLAG
  FROM
  `pbm-mac-lp-prod-ai.ds_sandbox.costplus_jan_2025`
    WHERE BRND_GNRC_CD = 'GNRC'
-- AND LOAD_DT = (SELECT MAX(LOAD_DT) FROM  `pbm-mac-lp-prod-de.publish.COSTPLUS_DRUG_PRICE`)
GROUP BY GPI_CD
),

NADAC_TABLE AS (
SELECT * FROM `pbm-mac-lp-prod-ai.ds_sandbox.nadac_test_eva`
    ),

COST_VANTAGE_TABLE AS (
SELECT 
GPI14 AS GPI,
ROUND(CAST(Cost_Fee_per_Unit AS FLOAT64),4) AS CV_UNIT_PRICE 
FROM 
  `anbc-pss-prod.fdm_cnfv_pss_prod.gms_gms_pgm_prm_68_cst_vantage_cost_file`
WHERE
(CURRENT_DATE() BETWEEN eff_dt AND thru_dt)
QUALIFY ROW_NUMBER() OVER (PARTITION BY GPI14 ORDER BY EFF_DT DESC) = 1
),

final_table AS (
SELECT * FROM (SELECT
  claims_mac.CUSTOMER_ID,
  claims_mac.CLIENT_NAME,
  claims_mac.CLIENT_TYPE,
  claims_mac.guarantee_category,
  claims_mac.MEDD_COMM,
  claims_mac.REGION,
  claims_mac.MEASUREMENT,
  claims_mac.CHAIN_SUBGROUP AS CHAIN_GROUP,
  claims_mac.GPI,
  claims_mac.NDC,
  claims_mac.CLAIMS,
  CAST(claims_mac.TOT_AWP AS NUMERIC) AS TOT_AWP,
  CAST(claims_mac.TOT_QTY AS NUMERIC) AS TOT_QTY,
  CASE WHEN TRIM(claims_mac.guarantee_category) = 'TrueCost'
    THEN CAST(claims_mac.TOT_SPEND AS NUMERIC) + CAST(claims_mac.TOT_DISP_FEE AS NUMERIC) 
    ELSE CAST(claims_mac.TOT_SPEND AS NUMERIC) END AS TOT_SPEND,
  CAST(claims_mac.TOT_MBR_COST AS NUMERIC) AS TOT_MBR_COST,
  CAST(claims_mac.AWP_UNIT AS NUMERIC) AS AWP_UNIT,
  CASE WHEN claims_mac.CHAIN_GROUP = 'CMK_SPECIALTY' or claims_mac.MEASUREMENT = 'S%' THEN CONCAT(claims_mac.CUSTOMER_ID,"SX") ELSE claims_mac.MAC_LIST END AS MAC_LIST, 
  ROUND(COALESCE(grx_raw.weighted_mean,GRX_PRICE_BOB_CHAIN_MEAS, GRX_PRICE_BOB_CHAIN, 
                  grx_old.GOODRX_UNIT_PRICE, grx_old_bob.GOODRX_PRICE_BOB_CHAIN, 
                  grx_spec.GOODRX_UNIT_PRICE,grx_spec_bob.GOODRX_PRICE_BOB,
                  goodrx_cost_unit, goodrx_cost_unit_bob),4) AS VCML_GRX_PRICE,
  ROUND(CASE WHEN TRIM(claims_mac.guarantee_category) = 'TrueCost'
          THEN (COALESCE(claims_mac.CURR_MAC_PRC, 
          CASE WHEN claims_mac.CURR_MAC_PRC IS NULL THEN
            CASE WHEN claims_mac.MEASUREMENT NOT LIKE 'S%' THEN 
              CASE WHEN claims_mac.MEDD_COMM = "MEDD" THEN (1-0.55)*AWP_UNIT ELSE (1-0.43)*AWP_UNIT  END
            WHEN claims_mac.MEASUREMENT LIKE 'S%' THEN CAST(claims_mac.TOT_SPEND AS NUMERIC)*CAST(claims_mac.AWP_UNIT AS NUMERIC)/CAST(claims_mac.TOT_AWP AS NUMERIC) END END) * claims_mac.TOT_QTY + claims_mac.TOT_DISP_FEE)/claims_mac.TOT_QTY
        ELSE COALESCE(claims_mac.CURR_MAC_PRC,
        CASE WHEN claims_mac.CURR_MAC_PRC IS NULL THEN 
          CASE WHEN claims_mac.MEASUREMENT NOT LIKE 'S%' THEN
            CASE WHEN claims_mac.MEDD_COMM = "MEDD" THEN (1-0.55)*AWP_UNIT ELSE (1-0.43)*AWP_UNIT END
          WHEN claims_mac.MEASUREMENT LIKE 'S%' THEN CAST(claims_mac.TOT_SPEND AS NUMERIC)*CAST(claims_mac.AWP_UNIT AS NUMERIC)/CAST(claims_mac.TOT_AWP AS NUMERIC) END END)
    END ,4) AS CURR_MAC_PRC,
  spclt.SPCLT_DRUG_IND,
  claims_mac.RUN_DATE,
  claims_mac.PRICE_MUTABLE,
  COST_PLUS_TABLE.MCCP_UNIT_PRICE,
  spclt.gpi_nm AS GPI_NM,
  COALESCE(clms.COSTSAVER_CLIENT, "N") AS COSTSAVER_CLIENT,
  CASE WHEN claims_mac.CURR_MAC_PRC IS NULL THEN "NON_MAC" ELSE "MAC" END AS MAC_NONMAC,
  CASE WHEN claims_mac.MEASUREMENT = 'R30' THEN (NADAC_UNIT_PRICE * claims_mac.TOT_QTY + (8-1.85) * claims_mac.CLAIMS)/claims_mac.TOT_QTY
       WHEN claims_mac.MEASUREMENT = 'R90' THEN (NADAC_UNIT_PRICE * claims_mac.TOT_QTY + (11.95-1.85) * claims_mac.CLAIMS)/claims_mac.TOT_QTY
       WHEN claims_mac.MEASUREMENT = 'M30' THEN (NADAC_UNIT_PRICE * claims_mac.TOT_QTY + (18.46-1.85) * claims_mac.CLAIMS)/claims_mac.TOT_QTY
       ELSE NADAC_UNIT_PRICE END AS NADAC_UNIT_PRICE,
  CASE WHEN claims_mac.MEASUREMENT = 'R30' THEN (COST_VANTAGE_TABLE.CV_UNIT_PRICE * claims_mac.TOT_QTY + (8-1.85) * claims_mac.CLAIMS)/claims_mac.TOT_QTY
       WHEN claims_mac.MEASUREMENT = 'R90' THEN (COST_VANTAGE_TABLE.CV_UNIT_PRICE * claims_mac.TOT_QTY + (11.95-1.85) * claims_mac.CLAIMS)/claims_mac.TOT_QTY
       WHEN claims_mac.MEASUREMENT = 'M30' THEN (COST_VANTAGE_TABLE.CV_UNIT_PRICE * claims_mac.TOT_QTY + (18.46-1.85) * claims_mac.CLAIMS)/claims_mac.TOT_QTY
       ELSE COST_VANTAGE_TABLE.CV_UNIT_PRICE END AS CV_UNIT_PRICE,
       CAST(claims_mac.TOT_DISP_FEE AS NUMERIC) AS TOT_DISP_FEE,
       CAST(claims_mac.TOT_SPEND AS NUMERIC) AS TOT_ING_COST
FROM (
  SELECT
    client_info.*,
    claims_agg.* EXCEPT(CUSTOMER_ID,
      NDC_PRICE,
      GPI_PRICE),
      CASE WHEN (client_info.guarantee_category LIKE '%Vanilla%' AND CHAIN_SUBGROUP LIKE "%R90OK%") THEN 'R30'
           WHEN CHAIN_SUBGROUP = 'MCHOICE' THEN 'R30'
           WHEN CHAIN_SUBGROUP = 'CMK_SPECIALTY' or MEASUREMENT like 'S%' THEN 'R30'
      ELSE MEASUREMENT 
      END AS DUMMY_MEASUREMENT
  FROM (
    SELECT
      *,
      CASE
        WHEN CONCAT(CUSTOMER_ID, GPI) IN (SELECT DISTINCT CONCAT(CLIENT, GPI) FROM `pbm-mac-lp-prod-de.ds_pro_lp.ger_opt_mac_price_override` WHERE NDC LIKE "%*%") THEN "GPI OVERRIDE LIST"
        WHEN CONCAT(CUSTOMER_ID, NDC) IN (SELECT DISTINCT CONCAT(CLIENT, NDC) FROM `pbm-mac-lp-prod-de.ds_pro_lp.ger_opt_mac_price_override` WHERE NDC NOT LIKE "%*%") THEN "NDC OVERRIDE LIST"
        WHEN NDC IN (SELECT DISTINCT drug_id AS NDC FROM `pbm-mac-lp-prod-de.ds_pro_lp.gpi_change_exclusion_ndc`) THEN "EXCLUSION LIST"
        WHEN CONCAT(MAC_LIST, NDC) IN (SELECT DISTINCT CONCAT(MAC_LIST, NDC)FROM 
            full_mac WHERE NDC NOT LIKE "%*%") THEN "NDC PRICING"
      ELSE "PRICE MUTABLE"
    END AS PRICE_MUTABLE,
      CASE
        WHEN CHAIN_SUBGROUP IN ("CVSSP_R90OK",'CVS_R90OK','CVSSP','MCHOICE',"CMK_SPECIALTY") THEN "CVS"
        WHEN CHAIN_SUBGROUP IN ("WAG_R90OK") THEN "WAG"
        WHEN CHAIN_SUBGROUP IN ("NONPREF_OTH_R90OK","PREF_OTH") THEN "NONPREF_OTH"
        WHEN CHAIN_SUBGROUP IN ("WMT_R90OK") THEN "WMT"
        WHEN CHAIN_SUBGROUP IN ("HMA_R90OK") THEN "HMA"
        WHEN CHAIN_SUBGROUP IN ("CAR_R90OK") THEN "CAR"
        WHEN CHAIN_SUBGROUP IN ("ELE_R90OK") THEN "ELE"
        WHEN CHAIN_SUBGROUP IN ("EPC_R90OK") THEN "EPC"
        WHEN CHAIN_SUBGROUP IN ("GEN_R90OK") THEN "GEN"
      ELSE CHAIN_SUBGROUP
    END AS DUMMY_CHAIN_GROUP
    FROM (
      SELECT
        claims.*,
        mac_gpi.PRICE AS GPI_PRICE,
      CASE 
        WHEN NDC_PRICE IS NOT NULL THEN NDC_PRICE
        WHEN claims.MEASUREMENT LIKE "SPCLT" THEN curr_mac_spec.CURR_MAC_PRC 
        WHEN curr_mac_spec.CURR_MAC_PRC  IS NULL AND mac_gpi.PRICE IS NULL AND NDC_PRICE IS NULL THEN NULL
        ELSE mac_gpi.PRICE
      END  AS CURR_MAC_PRC
      FROM (
        SELECT
          claims_mapped.*,
          mac_ndc.PRICE AS NDC_PRICE
        FROM  claims_mapped
        LEFT JOIN mac_ndc
          ON
          claims_mapped.MAC_LIST = mac_ndc.MAC_LIST
          AND claims_mapped.GPI = mac_ndc.GPI
          AND claims_mapped.NDC = mac_ndc.NDC) claims
      LEFT JOIN  mac_gpi
      ON
        claims.MAC_LIST = mac_gpi.MAC_LIST
        AND claims.GPI = mac_gpi.GPI
      LEFT JOIN curr_mac_spec
      ON claims.CUSTOMER_ID = curr_mac_spec.CUSTOMER_ID
        AND claims.GPI = curr_mac_spec.GPI
        AND claims.NDC = curr_mac_spec.NDC)) claims_agg
  LEFT JOIN client_info
    ON claims_agg.CUSTOMER_ID = client_info.CUSTOMER_ID) claims_mac
  LEFT JOIN (SELECT * FROM `pbm-mac-lp-prod-de.ds_pro_lp.client_goodrx_data_gpi` 
             UNION DISTINCT 
             SELECT * FROM `anbc-pss-prod.de_gms_enrv_pss_prod.gms_aetna_client_goodrx_data_gpi`
            )
             grx_raw
    ON claims_mac.GPI = grx_raw.GPI
    AND claims_mac.CUSTOMER_ID = grx_raw.CUSTOMER_ID
    AND claims_mac.DUMMY_MEASUREMENT = grx_raw.MEASUREMENT
    AND claims_mac.DUMMY_CHAIN_GROUP = grx_raw.CHAIN_GROUP
  LEFT JOIN (SELECT GPI, MEASUREMENT, CHAIN_GROUP, AVG(weighted_mean) AS GRX_PRICE_BOB_CHAIN_MEAS FROM
            (SELECT * FROM `pbm-mac-lp-prod-de.ds_pro_lp.client_goodrx_data_gpi` 
             UNION DISTINCT 
             SELECT * FROM `anbc-pss-prod.de_gms_enrv_pss_prod.gms_aetna_client_goodrx_data_gpi`
            ) GROUP BY 1,2,3) grx_chain_meas
    ON claims_mac.GPI = grx_chain_meas.GPI
    AND claims_mac.DUMMY_MEASUREMENT = grx_chain_meas.MEASUREMENT
    AND claims_mac.DUMMY_CHAIN_GROUP = grx_chain_meas.CHAIN_GROUP
  LEFT JOIN (SELECT GPI, CHAIN_GROUP, AVG(weighted_mean) AS GRX_PRICE_BOB_CHAIN FROM
            (SELECT * FROM `pbm-mac-lp-prod-de.ds_pro_lp.client_goodrx_data_gpi` 
             UNION DISTINCT 
             SELECT * FROM `anbc-pss-prod.de_gms_enrv_pss_prod.gms_aetna_client_goodrx_data_gpi`
            ) GROUP BY 1,2) grx_chain
    ON claims_mac.GPI = grx_chain.GPI
    AND claims_mac.DUMMY_CHAIN_GROUP = grx_chain.CHAIN_GROUP
  LEFT JOIN `pbm-mac-lp-prod-ai.ds_sandbox.grx_raw_df` grx_old
    ON claims_mac.GPI = grx_old.GPI
    AND claims_mac.DUMMY_CHAIN_GROUP = grx_old.CHAIN_GROUP
    AND claims_mac.MEASUREMENT = grx_old.MEASUREMENT
    AND claims_mac.RUN_DATE = grx_old.RUN_DATE
  LEFT JOIN (SELECT GPI, CHAIN_GROUP, RUN_DATE, MIN(GOODRX_UNIT_PRICE) AS GOODRX_PRICE_BOB_CHAIN FROM `pbm-mac-lp-prod-ai.ds_sandbox.grx_raw_df` GROUP  BY 1,2,3) grx_old_bob
    ON claims_mac.GPI = grx_old_bob.GPI
    AND claims_mac.DUMMY_CHAIN_GROUP = grx_old_bob.CHAIN_GROUP
    AND claims_mac.RUN_DATE = grx_old_bob.RUN_DATE
  LEFT JOIN `pbm-mac-lp-prod-ai.ds_sandbox.grx_raw_df` grx_spec
    ON claims_mac.GPI = grx_spec.GPI
    AND claims_mac.DUMMY_CHAIN_GROUP = grx_spec.CHAIN_GROUP
    AND claims_mac.DUMMY_MEASUREMENT = grx_spec.MEASUREMENT
    AND claims_mac.RUN_DATE = grx_spec.RUN_DATE
  LEFT JOIN (SELECT GPI, CHAIN_GROUP, RUN_DATE, MIN(GOODRX_UNIT_PRICE) AS GOODRX_PRICE_BOB FROM `pbm-mac-lp-prod-ai.ds_sandbox.grx_raw_df` GROUP  BY 1,2,3) grx_spec_bob
    ON claims_mac.GPI = grx_spec_bob.GPI
    AND claims_mac.DUMMY_CHAIN_GROUP = grx_spec_bob.CHAIN_GROUP
    AND claims_mac.RUN_DATE = grx_spec_bob.RUN_DATE
  LEFT JOIN `pbm-mac-lp-prod-ai.ds_sandbox.grx_flat_file` grx_flat_file
    ON claims_mac.GPI = grx_flat_file.GPI
    AND claims_mac.DUMMY_CHAIN_GROUP = grx_flat_file.CHAIN_GROUP
    AND claims_mac.DUMMY_MEASUREMENT = grx_flat_file.MEASUREMENT
  LEFT JOIN (SELECT GPI, CHAIN_GROUP, MIN(goodrx_cost_unit) AS goodrx_cost_unit_bob FROM `pbm-mac-lp-prod-ai.ds_sandbox.grx_flat_file` GROUP  BY 1,2) grx_flat_file_bob
    ON claims_mac.GPI = grx_flat_file_bob.GPI
    AND claims_mac.DUMMY_CHAIN_GROUP = grx_flat_file_bob.CHAIN_GROUP
  LEFT JOIN spclt
    ON claims_mac.GPI = spclt.GPI
  LEFT JOIN COST_PLUS_TABLE
    ON claims_mac.GPI = COST_PLUS_TABLE.GPI
  LEFT JOIN NADAC_TABLE
    ON claims_mac.GPI = NADAC_TABLE.GPI
    AND claims_mac.NDC = NADAC_TABLE.NDC
    AND claims_mac.CUSTOMER_ID = NADAC_TABLE.CUSTOMER_ID
  LEFT JOIN COST_VANTAGE_TABLE
    ON claims_mac.GPI = COST_VANTAGE_TABLE.GPI
  LEFT JOIN (SELECT
    DISTINCT CUSTOMER_ID,
    "Y" AS COSTSAVER_CLIENT
    FROM
    `anbc-pss-prod.de_gms_enrv_pss_prod.clms_survey_summary`) clms
    ON claims_mac.CUSTOMER_ID = clms.CUSTOMER_ID
WHERE claims_mac.CUSTOMER_ID IS NOT NULL
))


SELECT * FROM final_table
WHERE CURR_MAC_PRC IS NOT NULL
AND SPCLT_DRUG_IND IS NOT NULL
-- )
"""

In [ ]:
# bq.query(vcml_grx_mac_query)

In [ ]:
# TRUNCATE TABLE `pbm-mac-lp-prod-ai.ds_sandbox.vcml_grx_mac`;
# INSERT INTO `pbm-mac-lp-prod-ai.ds_sandbox.vcml_grx_mac` (
# SELECT * FROM `pbm-mac-lp-prod-ai.ds_sandbox.vcml_grx_mac_archive`
# WHERE RUN_DATE IN (SELECT MAX(RUN_DATE) FROM `pbm-mac-lp-prod-ai.ds_sandbox.vcml_grx_mac_archive`))